<a href="https://colab.research.google.com/github/Pluze/DeepLabCut/blob/main/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut for your standard (single animal) projects!

Some useful links:

- [DeepLabCut's GitHub: github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Single Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)


This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

### This notebook assumes you already have a project folder with labeled data!

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

As the COLAB environments were updated to CUDA 12.X and Python 3.11, we need to install DeepLabCut and TensorFlow in a distinct way to get TensorFlow to connect to the GPU.

In [ ]:
# this will take a couple of minutes to install all the dependencies!
!pip install --pre deeplabcut

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)** You will see this button at the output of the cells above ^.

In [ ]:
import deeplabcut

Loading DLC 3.0.0rc10...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your project folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [ ]:
# Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
# (We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: `/content/drive/My Drive/yourProjectFolderName`


In [8]:
# PLEASE EDIT THIS:
project_folder_name = "ratrun2-ziyu-2025-09-14"
video_type = "mp4" #, mp4, MOV, or avi, whatever you uploaded!

# No need to edit this, we are going to assume you put videos you want to analyze
# in the "videos" folder, but if this is NOT true, edit below:
videofile_path = [f"/content/drive/MyDrive/{project_folder_name}/videos/"]
print(videofile_path)

# The prediction files and labeled videos will be saved in this `labeled-videos` folder
# in your project folder; if you want them elsewhere, you can edit this;
# if you want the output files in the same folder as the videos, set this to an empty string.
destfolder = f"/content/drive/MyDrive/{project_folder_name}/labeled-videos"

#No need to edit this, as you set it when you passed the ProjectFolderName (above):
path_config_file = f"/content/drive/MyDrive/{project_folder_name}/config.yaml"
print(path_config_file)

# This creates a path variable that links to your Google Drive project

['/content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/']
/content/drive/MyDrive/ratrun2-ziyu-2025-09-14/config.yaml


## Create a training dataset:

### You must do this step inside of Colab

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models-pytorch** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pytorch_config.yaml**.

Now it is the time to start training the network!

In [9]:
# There are many more functions you can set here, including which network to use!
# Check the docstring for `create_training_dataset` for all options you can use!
superanimal_name = "superanimal_quadruped"

from deeplabcut.modelzoo import build_weight_init

weight_init = build_weight_init(
    cfg=path_config_file,
    super_animal=superanimal_name,
    model_name="resnet_50",                     # SuperAnimal-Quadruped 的 PyTorch 姿态模型
    detector_name="fasterrcnn_resnet50_fpn_v2", # 搭配的检测器
    with_decoder=False                          # 按官方示例保持 False
)

deeplabcut.create_training_dataset(
    path_config_file,
    engine=deeplabcut.Engine.PYTORCH,
    net_type="resnet_50",
    weight_init=weight_init,
    augmenter_type="albumentations",
)

[(0.95,
  1,
  (array([120, 125, 209, 224,  54, 295,  22, 192,  55, 115, 141,  23,   5,
          226,  46,  93,  31, 220, 198, 244,  25,  53,  69, 159,  98, 136,
          169, 128, 163, 164,  43,  62, 171,  18, 254, 269,   6, 153,  63,
          196, 121, 104, 259,  45,  65, 279, 317,  42, 161, 248,  66, 276,
          274,  72, 116,  26, 114,  85, 315, 147, 142, 186, 227,  52, 250,
            2,   1, 232, 201, 294,  24, 235, 124, 156,  84, 197,  39, 262,
          140, 260, 308, 270, 215, 231,  89,  67, 151, 189, 190, 139,  14,
          108, 289,  38,  19,  30, 290, 310, 256,  12, 247,  97, 319,   9,
          212, 211, 130, 182,  35, 299, 137, 245, 307,  36, 273,   4, 112,
          176, 208,  76, 234, 313, 312, 184,  61,  47, 148, 219,  92, 109,
            0,  40,  75, 152,   7,  59, 111, 126, 287, 218,  56,   8, 217,
           86, 230,  51, 238, 199,  37, 118, 119,  81,  32, 200, 100,  90,
          214, 174, 253, 282, 167, 318, 178, 241, 154, 268, 255, 145, 286,
          13

## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [10]:
# Let's also change the display and save_epochs just in case Colab takes away
# the GPU... If that happens, you can reload from a saved point using the
# `snapshot_path` argument to `deeplabcut.train_network`:
#   deeplabcut.train_network(..., snapshot_path="/content/.../snapshot-050.pt")

# Typically, you want to train to ~200 epochs. We set the batch size to 8 to
# utilize the GPU's capabilities.

# More info and there are more things you can set:
#   https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html#g-train-the-network

deeplabcut.train_network(
    path_config_file,
    save_epochs=5,
    batch_size=32,
    epochs=200,
    superanimal_name=superanimal_name,
    superanimal_transfer_learning=True,  # True=迁移学习（新解码层，点数不必与 SA 模型一致）
    # superanimal_transfer_learning=False  # False=微调，需与 SA 模型关键点数一致（Quadruped=39个）
)

# This will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end.

Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
metadata:
  project_path: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14
  pose_config_path: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/dlc-models-pytorch/iteration-0/ratrun2Sep14-trainset95shuffle1/train/pytorch_config.yaml
  bodyparts: ['iliac_crest', 'hip', 'knee', 'ankle', 'foot']
  unique_bodyparts: []
  individuals: ['animal']
  with_identity: None
method: bu
model:
  backbone:
    type: ResNet
    model_name: resnet50_gn
    output_stride: 16
    freeze_bn_stats: False
    freeze_bn_weights: False
  backbone_output_channels: 2048
  heads:
    bodypart:
      type: HeatmapHead
      weigh

KeyboardInterrupt: 

Note, that **when you hit "STOP" you will get a `KeyboardInterrupt` "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**

In [11]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as
# good as the labeler, so be sure your labels are good!


Evaluation scorer: DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60


100%|██████████| 16/16 [00:01<00:00, 13.86it/s]


Evaluation results file: DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60-results.csv
Evaluation results for DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60-results.csv (pcutoff: 0.6):
train rmse              1.48
train rmse_pcutoff      1.46
train mAP             100.00
train mAR             100.00
test rmse               1.80
test rmse_pcutoff       1.62
test mAP              100.00
test mAR              100.00
Name: (0.95, 1, 60, -1, 0.6), dtype: float64


## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [14]:
deeplabcut.analyze_videos(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

Creating the output folder /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos
Successfully loaded snapshot: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/dlc-models-pytorch/iteration-0/ratrun2Sep14-trainset95shuffle1/train/snapshot-best-060.pt
Analyzing videos with /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/dlc-models-pytorch/iteration-0/ratrun2Sep14-trainset95shuffle1/train/snapshot-best-060.pt
Using scorer: DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60
Starting to analyze /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/06ch6.mp4
Video metadata: 
  Overall # of frames:    600
  Duration of video [s]:  5.01
  fps:                    119.88
  resolution:             w=1920, h=1080

Running pose prediction with batch size 8


100%|██████████| 600/600 [00:43<00:00, 13.64it/s]


Saving results in /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/06ch6DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60.h5 and /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/06ch6DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60_full.pickle
Starting to analyze /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/02ch5.mp4
Video metadata: 
  Overall # of frames:    600
  Duration of video [s]:  5.01
  fps:                    119.88
  resolution:             w=1920, h=1080

Running pose prediction with batch size 8


100%|██████████| 600/600 [00:43<00:00, 13.69it/s]


Saving results in /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/02ch5DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60.h5 and /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/02ch5DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60_full.pickle
Starting to analyze /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/01ch5.mp4
Video metadata: 
  Overall # of frames:    600
  Duration of video [s]:  5.01
  fps:                    119.88
  resolution:             w=1920, h=1080

Running pose prediction with batch size 8


...

100%|██████████| 600/600 [00:44<00:00, 13.52it/s]


Saving results in /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/13pt6DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60.h5 and /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/13pt6DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60_full.pickle
Starting to analyze /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/14pt6.mp4
Video metadata: 
  Overall # of frames:    600
  Duration of video [s]:  5.01
  fps:                    119.88
  resolution:             w=1920, h=1080

Running pose prediction with batch size 8


100%|██████████| 600/600 [00:44<00:00, 13.59it/s]


Saving results in /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/14pt6DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60.h5 and /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/labeled-videos/14pt6DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.



'DLC_Resnet50_ratrun2Sep14shuffle1_snapshot_best-60'

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [15]:
deeplabcut.plot_trajectories(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

Analyzing all the videos in the directory...
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/11pt5.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/03ch5.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/09pt5.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/07intact.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/16sham.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/13pt6.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/01ch5.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/05ch6.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/02ch5.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/06ch6.mp4 and data.
Loading  /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/04ch6.mp4 and data.
Loading  /content/drive/MyDrive/ratrun

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [16]:
deeplabcut.create_labeled_video(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

Analyzing all the videos in the directory...
Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/13pt6.mp4Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/01ch5.mp4Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/16sham.mp4Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/15sham.mp4Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/05ch6.mp4Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/04ch6.mp4


Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/12pt6.mp4
Starting to process video: /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/06ch6.mp4Loading /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/01ch5.mp4 and data.
Loading /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/13pt6.mp4 and data.Loading /content/drive/MyDrive/ratrun2-ziyu-2025-09-14/videos/16sh

  0%|          | 0/600 [00:00<?, ?it/s]

Generating frames and creating video.


  0%|          | 0/600 [00:00<?, ?it/s]

Duration of video [s]: 5.0, recorded with 120 fps!
Overall # of frames: 600 with cropped frame dimensions: 1920 1080
Generating frames and creating video.Duration of video [s]: 5.0, recorded with 120 fps!
Overall # of frames: 600 with cropped frame dimensions: 1920 1080

Generating frames and creating video.


...

Overall # of frames: 600 with cropped frame dimensions: 1920 1080


 99%|█████████▉| 593/600 [00:26<00:00, 30.43it/s]

Generating frames and creating video.


100%|██████████| 600/600 [00:12<00:00, 47.55it/s]


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]